In [ ]:
from metrics.product_space_graph import ProductSpaceGraph
import pickle
import pandas as pd
import numpy as np
omegas = ["0.55", "0.6", "0.65"]

## Gráficos de resultados sobre Chile y Corea del Sur

In [ ]:
!source .venv/bin/activate && cd experimentacion && source replica.sh

In [ ]:
graphs = [
    ProductSpaceGraph(f"experimentacion/local-{omega}/exports_history.pkl")
    for omega in omegas
]

In [ ]:
graphs[0].plot("CHL", "Proceso de difusión con criterio local en Chile, Omega = 0.55")

In [ ]:
graphs[1].plot("CHL", "Proceso de difusión con criterio local en Chile, Omega = 0.6")

In [ ]:
graphs[2].plot("CHL", "Proceso de difusión con criterio local en Chile, Omega = 0.65")

In [ ]:
graphs[0].plot("KOR", "Proceso de difusión con criterio local en Corea del Sur, Omega = 0.55")

In [ ]:
graphs[1].plot("KOR", "Proceso de difusión con criterio local en Corea del Sur, Omega = 0.6")

In [ ]:
graphs[2].plot("KOR", "Proceso de difusión con criterio local en Corea del Sur, Omega = 0.65")

## Simulación a mano

Para chequear que no sea un problema con la simulación en DEVS, repito la simulación usando numpy.

In [ ]:
from models.product_space import exports_to_phi
exports = graphs[0].exports_by_generation.loc[(0, "CHL")].exports
phi = exports_to_phi(graphs[0].X_by_generation(0))
phi

In [ ]:
def iterate(exports, omega):
    return exports | (1 * ((phi @ np.diagflat(exports)).max(1) >= omega))

# dist[i] = max_j(phi[i,j] * exports[j]
#dist = [
#    max([phi[i][j] * exports[j] for j in range(exports.shape[0])
#        ])
#    for i in range(exports.shape[0])
#]
#((dist - (phi @ np.diagflat(exports)).max(1)) < 0.0001).all()

In [ ]:
simu = [exports]
for _ in range(5):
    simu.append(iterate(simu[-1], 0.55))
[sim.sum() for sim in simu]

Encuentro que haciendo la simulación manualmente, los resultados son al menos similares, y lejamos a los de Hidalgo (donde con $omega=0.55$ Chile lograba difundir bastante).

## Reproducción de otros gráficos

Intento reproducir otros gráficos de la publicación anteriores a la simulación.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
def largest_comp(omega):
    G = nx.from_numpy_matrix(1* (phi > omega))
    return len(max(nx.connected_components(G), key=len)) / len(G)

In [ ]:
omegas = [x/100 for x in range(30, 81, 5)]

plt.plot(omegas, list(map(largest_comp, omegas)))
plt.grid()

Noto que la caida se da $0.1$ antes que en la figura 3E original.

In [ ]:
data = pickle.load(open("data/stage1_data.pkl", "rb"))
data["X_matrices"][2000]

In [ ]:
def f(year):
    def largest_comp(omega):
        G = nx.from_numpy_matrix(1* (exports_to_phi(data["X_matrices"][year]) > omega))
        return len(max(nx.connected_components(G), key=len)) / len(G)
    return largest_comp

plt.plot(omegas, list(map(f(1998), omegas)), label="1998")
plt.plot(omegas, list(map(f(1999), omegas)), label="1999")
plt.plot(omegas, list(map(f(2000), omegas)), label="2000")
plt.legend()
plt.grid()

In [ ]:
def largest_comp(omega):
    X = (data["X_matrices"][1998] + data["X_matrices"][1999] + data["X_matrices"][2000])/3
    G = nx.from_numpy_matrix(1* (exports_to_phi(X) > omega))
    return len(max(nx.connected_components(G), key=len)) / len(G)

plt.plot(omegas, list(map(largest_comp, omegas)))
plt.grid()

In [ ]:
def links_by_threshold(t):
    return (1* (phi > t)).sum()
ts = list(map(lambda x: x/200, range(0, 200)))
plt.loglog(ts, list(map(links_by_threshold, ts)))

In [ ]:
plt.xscale('log')
plt.hist(phi.ravel(),bins=10.0**(np.arange(-3,0, 0.1)))